In [8]:
import plotly.express as px
import numpy as np
import pandas as pd
from faker import Faker
from dash import dcc, Dash, html
from dash.dependencies import Input, Output
import geopandas as gpd
from shapely.geometry import Point, mapping
import json


In [7]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur

In [ ]:

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 5000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Generate random opacity values for each row in the DataFrame
opacity = np.random.uniform(0.1, 0.9, size=num_rows)

# Add opacity values to the DataFrame
practice_df['opacity'] = opacity

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200 / 81000,  # ~200 meters in degrees
    'inside': 150 / 81000,   # ~150 meters in degrees
    'difficult': 125 / 81000 # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Display the first few rows of the DataFrame
print(practice_df.head(15))

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
map_with_aed_points = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Initialize the Dash app
app = Dash(__name__, prevent_initial_callbacks=True)

# Create Days of week radio items
radioitems = dcc.RadioItems(
    id='day-radioitems',
    options=[{'label': day, 'value': day} for day in daysofweek_categories],
    value='Monday',  # Default selected value
    inline=True
)

# Define the layout of the app
app.layout = html.Div([
    radioitems,
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    dcc.Graph(
        id='map-with-aed-points',
        figure=map_with_aed_points
    )
])

# Define callback to update the map based on the radio items
@app.callback(
    Output('map-with-aed-points', 'figure'),
    [Input('day-radioitems', 'value'),
     Input('time-slider', 'value')]
)
def update_map(selected_day, selected_hour):
    colors = []
    for _, row in practice_df.iterrows():
        days = row['daysofweek']
        opening_hour = row['opening_hour']
        closing_hour = row['closing_hour']
        if selected_day in days:
            if opening_hour <= selected_hour <= closing_hour:
                colors.append('green')  # Set color to green if selected hour is within opening and closing hours
            else:
                colors.append('red')  # Set color to red if selected hour is outside opening and closing hours
        else:
            colors.append('red')  # Set color to red if day is not selected

    # Create the updated map
    updated_map = px.scatter_mapbox(
        practice_df,
        lat='Latitude',
        lon='Longitude',
        color=colors,
        size='buffer_distance',
        size_max=15,
        zoom=zoom_level
    )

    # Update the layout to use a Mapbox style
    updated_map.update_layout(
        mapbox_style="open-street-map"
    )

    # Update marker properties to keep the same size while zooming
   

    return updated_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

     opacity                                         daysofweek  opening_hour  \
0   0.761434                              [Wednesday, Thursday]             7   
1   0.522375                                         [Thursday]             8   
2   0.402858                                          [Tuesday]             8   
3   0.134695                                  [Tuesday, Friday]             7   
4   0.500567                        [Wednesday, Friday, Sunday]             8   
5   0.717217                                          [Tuesday]             8   
6   0.468360  [Wednesday, Saturday, Tuesday, Friday, Thursda...             8   
7   0.648421        [Saturday, Tuesday, Friday, Monday, Sunday]             8   
8   0.402698                  [Monday, Sunday, Friday, Tuesday]             8   
9   0.875524     [Friday, Wednesday, Tuesday, Monday, Saturday]             8   
10  0.894298  [Thursday, Wednesday, Tuesday, Monday, Friday,...             8   
11  0.580021  [Monday, Satur